# Causalvis: Visualizations for Causal Inference

In [2]:
import json
import pandas as pd

### Load Data

In [3]:
data = pd.read_csv('./data/student-mat.csv', delimiter=';')
data.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [4]:
drop_col = ['school','sex','age','Mjob', 'Fjob','reason','guardian']
data = data.drop(columns=drop_col)
data.head(5)

,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,U,GT3,A,4,4,2,2,0,yes,no,...,4,3,4,1,1,3,6,5,6,6
1,U,GT3,T,1,1,1,2,0,no,yes,...,5,3,3,1,1,3,4,5,5,6
2,U,LE3,T,1,1,1,2,3,yes,no,...,4,3,2,2,3,3,10,7,8,10
3,U,GT3,T,4,2,1,3,0,no,yes,...,3,2,2,1,1,5,2,15,14,15
4,U,GT3,T,3,3,1,2,0,no,yes,...,4,3,2,1,2,5,4,6,10,10


### Constructing the DAG

In [5]:
import numpy as np

struct_data = data.copy()
non_numeric_columns = list(struct_data.select_dtypes(exclude=[np.number]).columns)

print(non_numeric_columns)

['address', 'famsize', 'Pstatus', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']


In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in non_numeric_columns:
    struct_data[col] = le.fit_transform(struct_data[col])

struct_data.head(5)

,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,1,0,0,4,4,2,2,0,1,0,...,4,3,4,1,1,3,6,5,6,6
1,1,0,1,1,1,1,2,0,0,1,...,5,3,3,1,1,3,4,5,5,6
2,1,1,1,1,1,1,2,3,1,0,...,4,3,2,2,3,3,10,7,8,10
3,1,0,1,4,2,1,3,0,0,1,...,3,2,2,1,1,5,2,15,14,15
4,1,0,1,3,3,1,2,0,0,1,...,4,3,2,1,2,5,4,6,10,10


### Editing the DAG

You can pass all the attributes in the dataset to the DAG widget and construct your own DAG.
If you would like to skip this step, you can load the example from the saved DAG file in the next cell.

In [7]:
from causalvis import DAG

DAG(data=struct_data)

DAG(component='DAG', props={'attributes': ['absences', 'activities', 'address', 'Dalc', 'failures', 'famrel', …

### Identifying control variables

Once we are satisfied with the DAG, we can set the treatment and outcome variables. In this case, we are interested in `G1` as the outcome variable, while `absences` is the treatment variable of interest. After selecting these variables, we see that other attributes in the DAG are automatically colored to reflect their relationship to the treatment and outcome.

From the downloads button, we can get a list of the `confounds` and `prognostic` factors to control for.

In [8]:
with open('./data/DAG_student_example (1).json', 'r') as d:
    graph = json.load(d)

In [9]:
dg = DAG(graph=graph)
dg

DAG(component='DAG', props={'attributes': None, 'graph': {'nodes': [{'x': 60.76065021003316, 'y': 411.32658125…

In [13]:
confounds = [c['name'] for c in dg.confounds]
confounds

['Pstatus', 'failures', 'internet', 'health', 'famsup', 'Medu']

In [14]:
prognostics = [p['name'] for p in dg.prognostics]
prognostics

['studytime', 'schoolsup', 'paid', 'higher']

# Cohort Construction and Refinement

In [15]:
from causalvis import CohortEvaluator

In [16]:
data.shape

(395, 26)

In [17]:
a = data["absences"]
a = a >= a.median()
a = a.astype(int)

In [18]:
y = data["G1"]

In [19]:
covariates = confounds + prognostics

X = data[covariates]
X.dtypes

Pstatus      object
failures      int64
internet     object
health        int64
famsup       object
Medu          int64
studytime     int64
schoolsup    object
paid         object
higher       object
dtype: object

In [20]:
X = data.loc[:, covariates]
X = pd.get_dummies(X, drop_first=True)
X.shape

(395, 10)

In [21]:
from sklearn.linear_model import LogisticRegression
from causallib.estimation import IPW

lr = LogisticRegression(solver='lbfgs', max_iter=1000)
ipw = IPW(lr)

/Users/grace/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/grace/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [22]:
ipw.fit(X, a)

IPW(clip_max=None, clip_min=None, use_stabilized=False, verbose=False,
    learner=LogisticRegression(max_iter=1000))

In [23]:
propMatrix = ipw.compute_propensity_matrix(X, a).to_dict(orient="records")

In [24]:
def formatData(X, a, y, propMatrix):
    unadjustedData = []

    confounds = X.to_dict(orient="records")
    a.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)

    for i in range(len(confounds)):
        newDataInstance = confounds[i]
        newDataInstance['treatment'] = a[i]
        newDataInstance['propensity'] = propMatrix[i]

        unadjustedData.append(newDataInstance)
        
    return unadjustedData

# IPW

In [43]:
ceval = CohortEvaluator(unadjustedCohort=formatData(X, a, y, propMatrix))
ceval

CohortEvaluator(component='CohortEvaluator', props={'unadjustedCohort': [{'failures': 0, 'health': 3, 'Medu': …

Since the population is well balanced, we can go ahead and estimate the treatment effect. This can be done quite easily in causallib, using the `estimate_population_outcome` function. From the result, we can see that having < 4 absences a year causes a slightly higher score for `G1`.

In [27]:
ipw.estimate_population_outcome(X, a, y)

0    11.036720
1    10.797529
dtype: float64

# Matching

In [28]:
from causallib.estimation import IPW, Matching
from causallib.preprocessing.transformers import PropensityTransformer
from causallib.preprocessing.transformers import MatchingTransformer
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [29]:
matcher = MatchingTransformer(with_replacement=True, n_neighbors=1, caliper=2)
matcher.fit(X, a, y)
Xm, am, ym = matcher.transform(X, a, y)

Xm.reset_index(drop=True, inplace=True)
am.reset_index(drop=True, inplace=True)
ym.reset_index(drop=True, inplace=True)

def learner(): return LogisticRegression(solver="liblinear",
                                         max_iter=5000,
                                         class_weight="balanced")

ipw = IPW(learner=learner())
ipw.fit(Xm, am)
propMatrixm = ipw.compute_propensity_matrix(Xm, am).to_dict(orient="records")

In [30]:
Xm.shape

(328, 10)

In [31]:
from causalvis import CohortEvaluator

ceval = CohortEvaluator(adjustedCohort=formatData(Xm, am, ym, propMatrixm), unadjustedCohort=formatData(X, a, y, propMatrix))
ceval

CohortEvaluator(component='CohortEvaluator', props={'unadjustedCohort': [{'failures': 0, 'health': 3, 'Medu': …

In [32]:
matcher = MatchingTransformer(with_replacement=True, n_neighbors=1, caliper=1)
matcher.fit(X, a, y)
Xm2, am2, ym2 = matcher.transform(X, a, y)

Xm2.reset_index(drop=True, inplace=True)
am2.reset_index(drop=True, inplace=True)
ym2.reset_index(drop=True, inplace=True)

def learner(): return LogisticRegression(solver="liblinear",
                                         max_iter=5000,
                                         class_weight="balanced")

ipw = IPW(learner=learner())
ipw.fit(Xm2, am2)
propMatrixm2 = ipw.compute_propensity_matrix(Xm2, am2).to_dict(orient="records")

In [33]:
Xm2.shape

(200, 10)

In [34]:
ceval2 = CohortEvaluator(adjustedCohort=formatData(Xm2, am2, ym2, propMatrixm2), unadjustedCohort=formatData(X, a, y, propMatrix))
ceval2

CohortEvaluator(component='CohortEvaluator', props={'unadjustedCohort': [{'failures': 0, 'health': 3, 'Medu': …

In [35]:
propensity_transform = PropensityTransformer(
    include_covariates=False, learner=learner())
matcher = Matching(propensity_transform=propensity_transform,
                   with_replacement=True, n_neighbors=1, knn_backend="sklearn")
matcher.fit(Xm2, am2, ym2)
matches = matcher._execute_matching(Xm2, am2)

In [36]:
matcher.estimate_population_outcome(Xm2, am2)

0    11.140
1    11.565
dtype: float64

In [37]:
population_outcome = matcher.estimate_population_outcome(Xm2, am2)
ATE = population_outcome[1] - population_outcome[0]
ATE

0.42499999999999893

In [38]:
ind_outcomes = matcher.estimate_individual_outcome(Xm2, am2).to_dict(orient="records")

ind_effects = []

for i in ind_outcomes:
    i_treatment = i[1]
    i_control = i[0]
    
    effect = i_treatment - i_control
    
    ind_effects.append(effect)

In [39]:
data = []
Xm_dict = Xm2.to_dict(orient="records")

for i in range(len(Xm_dict)):
    newDataInstance = Xm_dict[i]
    newDataInstance['treatment'] = int(am2[i])
    newDataInstance['outcome'] = int(ym2[i])
    newDataInstance['effect'] = int(ind_effects[i])
    data.append(newDataInstance)

In [40]:
from causalvis import TreatmentEffectExplorer

TreatmentEffectExplorer(data=data)

TreatmentEffectExplorer(component='TreatmentEffectExplorer', props={'data': [{'failures': 0, 'health': 3, 'Med…

In [41]:
from causalvis import VersionHistory

vh = VersionHistory()
vh

VersionHistory(component='VersionHistory', props={'versions': [], 'effect': 'effect'})

In [42]:
vh.addVersion((dg.DAG, Xm2, ATE.tolist()))

In [124]:
vh.saveVersions()